In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

E: Package 'python-software-properties' has no installation candidate


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/'My Drive'/'CIS 520 Project'

/content/drive/My Drive/CIS 520 Project


In [0]:
#imports
import warnings
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.decomposition import PCA
from matplotlib import pyplot
from collections import Counter
import seaborn as sns

In [0]:
def get_data(csvfile):
  df = pd.read_pickle(csvfile)
  x, y = df['text'].tolist(), df['stars_x'].tolist()
  return x, y

In [7]:
#imports 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Conv1D, GlobalMaxPooling1D,MaxPooling1D, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.naive_bayes import MultinomialNB 
import tensorflow as tf
tf.test.gpu_device_name()

Using TensorFlow backend.


'/device:GPU:0'

In [8]:
#preparing X and y!!!
csvfile = 'reviews_bow.csv'
reviews, stars = get_data(csvfile)
warnings.filterwarnings("ignore")
tempy, tempx = stars[:1000000], reviews[:1000000]
print('Data loaded')

#preparing y
y = np.array(tempy, dtype=np.uint8)
ymulti = y-1
y = to_categorical(ymulti, num_classes=5)
print('y prepared', y.shape)

#preparing x
embedding_size = 50
vocab_size = 20000 #will use most frequent 50k words
num_words = 200 #97%coverage for reviews
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(tempx)
sequences = tokenizer.texts_to_sequences(tempx)
padded_str = pad_sequences(sequences, maxlen = num_words)
#type changed here to use less memory 
x = np.array(padded_str, dtype=np.uint16)
# x = np.expand_dims(x, axis = 2)
print('x prepared', x.shape)

Data loaded
y prepared (1000000, 5)
x prepared (1000000, 200)


In [9]:
#class weights
# https://stackoverflow.com/questions/43481490/keras-class-weights-class-weight-for-one-hot-encoding
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)

{0: 2.0799101478816113, 1: 2.4022581226352773, 2: 1.598848828843233, 3: 0.7550874013667082, 4: 0.46443906211175795}


In [0]:
def CNN_simple():
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=num_words))
    model.add(Dropout(0.2))
    model.add(Conv1D(128, 11, strides=1, activation='relu'))#, input_shape=(num_words,1)))
    model.add(Conv1D(128, 7, strides=1, activation='relu'))
    model.add(GlobalMaxPooling1D())
    # model.add(Dropout(0.2))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.2))
    # model.add(Dense(120, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse','accuracy'])
    print(model.summary())
    return model

In [12]:
#CNN
#train-test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify = y)
print('X train shape: ', type(X_train), ' y train shape: ', type(y_train))
print('X train type: ', X_train.shape, ' y train type: ', y_train.shape)

model = CNN_simple()
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose = 1, class_weight=d_class_weights)
model.save('cnn_mse.h5')
scores = model.evaluate(X_test, y_test, verbose=0)
pred = model.predict_classes(X_test)
y_int = [np.argmax(r) for r in y_test]
print(scores)
print(metrics.accuracy_score(y_int, pred))
print(metrics.classification_report(y_int, pred))

X train shape:  <class 'numpy.ndarray'>  y train shape:  <class 'numpy.ndarray'>
X train type:  (800000, 200)  y train type:  (800000, 5)
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 50)           1000000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 200, 50)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 190, 128)          70528     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 184, 128)          114816    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)        